In [ ]:
pip install neo4j py2neo

In [ ]:
pip install pdfplumber

In [10]:
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
import pdfplumber
import re


In [3]:
# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

In [4]:
# Connessione al database
graph = Graph(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [7]:
file_path = "./Hackapizza Dataset/Misc/Manuale di Cucina.pdf"

In [9]:
def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Leggiamo il PDF
pdf_text = extract_text_from_pdf(file_path)

# Stampiamo un'anteprima
print(pdf_text[:500])  # Stampa i primi 500 caratteri


Introduzione
Io sono il grande Sirius Cosmo, lo chef stellare per eccellenza. Se non mi conoscete ancora, preparatevi:
il mio nome è sinonimo di cucina galattica. In questo manuale vi insegnerò tutto quello che serve per
diventare veri cuochi, da Alpha Centauri fino alla Nebulosa del Granchio (dove, a proposito, non hanno
neanche un crostaceo decente). Impareremo insieme le licenze e le abilità fondamentali per cucinare
nello spazio senza mandare in tilt lʼintero sistema di supporto vitale della


In [13]:
def extract_chapters_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"

    # Suddividiamo il testo nei capitoli basandoci su "CAPITOLO X"
    chapters = re.split(r'(Capitolo \d+)', text)
    
    structured_data = {}
    for i in range(1, len(chapters), 2):
        chapter_title = chapters[i].strip()
        chapter_content = chapters[i + 1].strip()
        structured_data[chapter_title] = chapter_content
    
    return structured_data

# Leggiamo il PDF e estraiamo i capitoli
pdf_chapters = extract_chapters_from_pdf(file_path)

# Stampiamo i titoli dei capitoli
print("📖 Capitoli trovati:")
for chapter in pdf_chapters.keys():
    print(f"- {chapter}")


📖 Capitoli trovati:
- Capitolo 1
- Capitolo 2
- Capitolo 3
- Capitolo 4
- Capitolo 5


In [20]:
def create_cooking_graph(chapters):
    # Nodo principale del manuale
    manual_node = Node("Document", name="Manuale di Cucina")
    graph.create(manual_node)

    for chapter, content in chapters.items():
        # Creiamo un nodo Capitolo
        chapter_node = Node("Chapter", name=chapter, content=content[:500])  # Solo un estratto
        graph.create(chapter_node)
        graph.create(Relationship(manual_node, "CONTAINS", chapter_node))

        # Se il capitolo riguarda tecniche di cottura, estraiamo concetti chiave
        if "3" in chapter or "4" in chapter or "5" in chapter:
            techniques = re.findall(r"\n([A-Z\s]+)\n", content)  # Titoli in maiuscolo
            for technique in techniques[:10]:  # Limitiamo a 10 tecniche per esempio
                technique_node = Node("Technique", name=technique.strip())
                graph.create(technique_node)
                graph.create(Relationship(chapter_node, "DESCRIBE", technique_node))

    print("✅ Grafo del manuale di cucina creato!")

# Creiamo il grafo
create_cooking_graph(pdf_chapters)

✅ Grafo del manuale di cucina creato!


In [21]:
query = """
MATCH (c:Chapter)-[:DESCRIBE]->(t:Technique)
WHERE c.name CONTAINS '3' OR c.name CONTAINS '4' OR c.name CONTAINS '5'
RETURN c.name, t.name
"""
results = graph.run(query)

print("🔥 Tecniche di Cottura:")
for record in results:
    print(f"- {record['t.name']} (dal {record['c.name']})")

🔥 Tecniche di Cottura:
